# Run experiments with CUIs. 

In [1]:
import xgboost as xgb
from xgboost import XGBClassifier

In [2]:
data_dir = "../data/"
import sys
sys.path.insert(0,"../tools")
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer as Cvec
from sklearn.feature_extraction.text import TfidfVectorizer as Tvec
from sklearn.ensemble import RandomForestClassifier as RFC
import copy
from tqdm import tqdm
import imp
import utils
inter_dir = "..//intermediate_result//"

## Load data

In [3]:
# previous version is merged_cui.csv; changed 0205
merged_df = pd.read_csv(inter_dir + "merged_cui_consolidated.csv",low_memory=False)


In [4]:
merged_df

,Unnamed: 0,ChartID,VISITIDCODE,Cerebrovascular disease present,Timing of Cerebrovascular disease,Cerebrovascular disease admit category,DXCODE1,DXCODE2,DXCODE3,DXCODE4,...,neurological_diagnostics,non-physician_progress_notes,nursing_notes,nursing_transfer_report,outpatient_consultation,pharmacy_care_plan,picc_~_midline_record,rehabilitation,social_work_assessment,urological_diagnostics
0,0,940,1.000390e+11,No,NaN,NaN,D351,E213,I100,NaN,...,,C0004448 C0004448 C0004448 C0004448 C0006147 C...,C0005767 C0005767 C0005767 C0005767 C0005767 C...,C0001268 C0001268 C0001268 C0001268 C0001268...,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1304,1.000390e+11,No,NaN,NaN,M179,NaN,NaN,NaN,...,,C0004448 C0004448 C0004448 C0006147 C0006147 C...,C0005767 C0005767 C0005767 C0005767 C0005767 C...,C0001268 C0001268 C0001268 C0001268 C0001268...,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1789,1.000393e+11,No,NaN,NaN,M179,J449,E119,I872,...,,C0002903 C0002903 C0002903 C0002903 C0002903 C...,C0005767 C0005767 C0005767 C0005767 C0005767 C...,C0001268 C0001268 C0001268 C0001268 C0001268...,NaN,NaN,NaN,NaN,NaN,NaN
3,3,906,1.000394e+11,No,NaN,NaN,M170,E119,I100,NaN,...,,C0002903 C0002903 C0004448 C0004448 C0004448 C...,C0005767 C0005767 C0005767 C0005767 C0005767 C...,C0001268 C0001268 C0001268 C0001268 C0001268...,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1500,1.000394e+11,No,NaN,NaN,M179,I100,NaN,NaN,...,,C0004448 C0013227 C0013227 C0013227 C0013227 C...,C0005767 C0005767 C0005767 C0005767 C0005767 C...,C0001268 C0001268 C0001268 C0001268 C0001268...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3031,3031,2552,1.000414e+11,No,NaN,NaN,S82800,S9340,W2208,U985,...,,C0004448 C0004448 C0004448 C0004448 C0004448 C...,C0005767 C0005767 C0005767 C0005767 C0005767 C...,C0005767 C0005823 C0009421 C0011209 C0011209 C...,NaN,NaN,NaN,NaN,NaN,NaN
3032,3032,3222,1.000415e+11,No,NaN,NaN,F101,F141,Z731,NaN,...,,C0004448 C0004448 C0004448 C0013227 C0013227 C...,C0005767 C0005767 C0005767 C0005767 C0005767 C...,C0005767 C0005823 C0011209 C0011209 C0011900 C...,C0001962 C0001962 C0001962 C0001962 C0001962 C...,NaN,NaN,NaN,NaN,NaN
3033,3033,502,1.000414e+11,No,NaN,NaN,I211,R9430,I2510,F172,...,,C0004448 C0013227 C0013227 C0013227 C0013227 C...,C0005767 C0005767 C0005767 C0005767 C0005767 C...,,NaN,NaN,NaN,NaN,NaN,NaN
3034,3034,1112,1.000415e+11,No,NaN,NaN,T814,L024,Y838,E1152,...,,C0004448 C0004448 C0004448 C0004448 C0006147 C...,C0005767 C0005767 C0005767 C0005767 C0005767 C...,C0005767 C0005823 C0009421 C0011209 C0011209 C...,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# total_txt is a list of string.
# Each text is a concatenation of all texts of a patient belonging to the selected types
# total_label is a list of label consists of 0 and 1. 1 means positive.
total_txt,total_label = utils.prep_cui(merged_df,selected_types,pos_ids,neg_ids)

NameError: name 'selected_types' is not defined

## prepare the log directory, log file and run models

In [8]:
os.mkdir('./exps/non_neg_CUI_xgboost_log')

FileExistsError: [WinError 183] Cannot create a file when that file already exists: './exps/non_neg_CUI_xgboost_log'

### Run exps is almost a wrapper of the "run models" function from utils. After the model runs, the combination of the vectorizer and model with the highest sensitivity is taken out and written to the log file together with the sensitivity.

In [5]:
imp.reload(utils)
candi_vec = [Cvec(stop_words = 'english',max_df = .8,min_df = 10),
             Tvec(stop_words = 'english',max_df = .8,min_df = 10)]
candi_models = [RFC(n_estimators = 200,random_state=221),RFC(random_state=221),XGBClassifier(random_state = 221,n_estimators = 200),XGBClassifier(random_state = 221)]
folder_name = './exps/non_neg_CUI_xgboost_log/'
def run_exps(selected_types):
    '''
    Pick the highest sensitivity and document it into the log
    '''
    f_log = open(folder_name + '{}.txt'.format(now_time),'a')
    f_log.writelines('\n' + str(selected_types) + ':')
    total_txt,total_label = utils.prep_cui(merged_df,selected_types,pos_ids,neg_ids)
    sens_dict = utils.run_models(total_txt,total_label,candi_vec,candi_models)
    best_result = {'vec':0,'model':0,'sens':-1}
    for vec,pair in sens_dict.items():
        for model,sens in pair.items():
            if (np.mean(sens) > best_result['sens']):
                best_result['vec'] = candi_vec[vec]
                best_result['model'] = candi_models[model]
                best_result['sens'] = np.mean(sens)
    f_log.writelines(str(best_result))
    f_log.close()
    return best_result['sens']

In [6]:
pos_ids,neg_ids = utils.get_group_ids(merged_df)

### The process of feed forward document search. "selected" is a vector that is initially all zeros with a length of the number of document types. The position corresponding to the document type selected during the search process will be marked as 1. It ends when the optimal 5 documents are found or there is no improvement after adding a new document.

In [7]:
from datetime import datetime
doc_types = list(merged_df.columns)[32:]
type_num = len(doc_types)
selected = np.zeros(type_num)
max_sens = 0
cur_sens = 0
cur_types = []
now_time = str(datetime.now())[5:16].replace(' ','_').replace(':','_')
while (np.sum(selected) < 5):
    cur_selected = -1
    for i in range(type_num):
        if (not selected[i]):
            tmp_types = copy.deepcopy(cur_types)
            tmp_types.append(doc_types[i])
            try:
                cur_sens = run_exps(tmp_types)
            except:
                cur_sens = 0
            if (cur_sens > max_sens):
                max_sens = cur_sens
                cur_selected = i
    if (cur_selected > 0):
        cur_types.append(doc_types[cur_selected])
        selected[cur_selected] = 1
    else:
        break

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
5it [00:40,  8.19s/it]
5it [01:35, 19.12s/it]
5it [00:21,  4.35s/it]
5it [00:28,  5.61s/it]
5it [00:13,  2.65s/it]
5it [00:18,  3.76s/it]
5it [00:45,  9.05s/it]
5it [01:25, 17.18s/it]
5it [00:10,  2.02s/it]
5it [03:02, 36.51s/it]
5it [07:12, 86.41s/it]
5it [01:26, 17.36s/it]
5it [00:11,  2.23s/it]
5it [00:33,  6.71s/it]
5it [00:09,  1.85s/it]
0it [00:00, ?it/s]
5it [00:07,  1.57s/it]
0it [00:00, ?it/s]
5it [01:15, 15.11s/it]
5it [01:17, 15.46s/it]
5it [01:19, 15.89s/it]
5it [01:21, 16.29s/it]
5it [01:18, 15.77s/it]
5it [01:20, 16.05s/it]
5it [01:34, 18.87s/it]
5it [02:16, 27.38s/it]
5it [01:22, 16.54s/it]
5it [01:32, 18.59s/it]
5it [01:27, 17.41s/it]
5it [01:32, 18.52s/it]
5it [01:43, 20.74s/it]
5it [02:09, 25.86s/it]
5it [01:22, 16.55s/it]
5it [03:17, 39.43s/it]
5it [07:20, 88.14s/it]
5it [01:24, 16.97s/it]
5it [01:40, 20.19s/it]
5it [01:18, 15.67s/it]
5it [01:20, 16.11s/i

In [8]:
print(selected)

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0.]
